In [1]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression
import xml.etree.ElementTree as ET
import csv
import json

In [2]:

ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
ET.register_namespace('xml','http://www.w3.org/XML/1998/namespace')

# Not all witnesses were enriched with reference identification.
# Change paths to fit your own folder organization.
witnesses = {
    "basnage" : [1678,"1678 - Basnage"],
    "berault" : [1614, "1614 - Bérault"],
    "merville" : [1731,"1731 - Merville"],
    "pesnelle" : [1771,"1771 - Pesnelle"],
    "terrien" : [1578,"1578 - Terrien"]
}
binpath = "/home/mpica/Progs/perso/CONDE/editions/base-version/"
einpath = "_base.xml"

outpath = "dataviz/2022-09-27/numbers.csv"

In [8]:
out_dict = []
for witness in witnesses.keys():
    
    wit_dict = {'Témoin' : witnesses[witness][1], 'Date' : witnesses[witness][0]}
    
    fullpath = binpath + witness + einpath
    
    authors = {}
    
    with open(fullpath) as xmlfile:
        tree = ET.parse(xmlfile)
        root = tree.getroot()
        
        for author in root.findall('.//{http://www.tei-c.org/ns/1.0}person'):
            # Get <birth> element.
            fullbirth = author.find('.//{http://www.tei-c.org/ns/1.0}birth')

            try:
                ident = "#" + author.get("{http://www.w3.org/XML/1998/namespace}id")
            except:
                print("Weird guy here, no ID.")

            try:
                if fullbirth.get('when') != None:
                    authors[ident] = {'birth' : int(fullbirth.get('when')), 'count' : 0, 'tokens' : 0}
                else:
                    authors[ident] = {'birth' : int(fullbirth.get('notBefore')), 'count' : 0, 'tokens' : 0}
            except:
                authors[ident] = {'birth' : 2022, 'count' : 0, 'tokens' : 0}
            #print(ident, authors[ident])
        
        wit_dict['Total des citations'] = len(root.findall('.//{http://www.tei-c.org/ns/1.0}text//{http://www.tei-c.org/ns/1.0}ref[@corresp]'))
        wit_dict['Nombre de tokens'] = len(root.findall('.//{http://www.tei-c.org/ns/1.0}w'))
        
        for ref in root.findall('.//{http://www.tei-c.org/ns/1.0}text//{http://www.tei-c.org/ns/1.0}ref[@corresp]'):
            corresp = ref.get('corresp')
            if corresp in authors.keys():
                authors[corresp]['count'] += 1
                authors[corresp]['tokens'] += len(ref.findall(".//{http://www.tei-c.org/ns/1.0}w"))
            else:
                continue
                
                
        classics = 0
        clatokens = 0
        for author in authors.keys():
            if authors[author]['birth'] <= 560:
                #print(author)
                classics += authors[author]['count']
                clatokens += authors[author]['tokens']
                
        wit_dict['Citations de "classiques"'] = classics
        wit_dict['Pourcentage de classiques'] = wit_dict['Citations de "classiques"'] * 100 / wit_dict['Total des citations']
        wit_dict['Rapport tokens/citations classiques'] = clatokens * 100 / wit_dict['Nombre de tokens']
        
        print(wit_dict)
        out_dict.append(wit_dict)

        
with open('dataviz/2022-09-28/tableau-recap.csv', 'w') as csvfile:
    csvwriting = csv.DictWriter(csvfile, fieldnames = out_dict[0].keys())
    csvwriting.writeheader()
    csvwriting.writerows(out_dict)

{'Témoin': '1678 - Basnage', 'Date': 1678, 'Total des citations': 5115, 'Nombre de tokens': 1524732, 'Citations de "classiques"': 496, 'Pourcentage de classiques': 9.696969696969697, 'Rapport tokens/citations classiques': 0.04105639548458352}
{'Témoin': '1614 - Bérault', 'Date': 1614, 'Total des citations': 1673, 'Nombre de tokens': 637525, 'Citations de "classiques"': 160, 'Pourcentage de classiques': 9.563658099222954, 'Rapport tokens/citations classiques': 0.02807733030077252}
{'Témoin': '1731 - Merville', 'Date': 1731, 'Total des citations': 68, 'Nombre de tokens': 564131, 'Citations de "classiques"': 3, 'Pourcentage de classiques': 4.411764705882353, 'Rapport tokens/citations classiques': 0.0007090551662645733}
{'Témoin': '1771 - Pesnelle', 'Date': 1771, 'Total des citations': 3170, 'Nombre de tokens': 713652, 'Citations de "classiques"': 61, 'Pourcentage de classiques': 1.9242902208201893, 'Rapport tokens/citations classiques': 0.008827832052597065}
{'Témoin': '1578 - Terrien', '